### OBTAINING PARK LAYER
The goal of this notebook is to produce a final urban green areas layer to performe accessibility analysis. <br>
I will use as a baseline the R03土地利用現況 file, which authored by Tokyo Metropolitan Government (opendata.metro.tokyo).The issue with these data is that they include also cemeteries and sports facilities as parks. My solution is the following:
- Extract from the land use data the parks (LU_1: 300)
- Query OSM for cemeteries, graveyeards, pitches and sport centers.
- First merge (dissolve) the extracted land park polygons
- Exclude the UGS polygons which overlap to the OSM layer


##### Import libraries and datasets

In [2]:
# TODO check where I set all the names to the same value
import os
import geopandas as gpd
import pandas as pd
import fiona
from shapely.geometry import Polygon

In [3]:
park_layer_datapath = os.path.join("..\\data\\provisional\\park_layer_data.gpkg")
landuse = gpd.read_file(park_layer_datapath, layer='land_use') # layer of Tokyo landuse
unwanted = gpd.read_file(park_layer_datapath, layer='unwanted_features') # layer of cemeteries, graveyeards and sport facilities

##### Preprocess parks data
- Fix CRS
- Merge adjecent parks
- Create index and recompute areas

In [4]:
print(f"original CRS: {landuse.crs}") # originally the data is in EPSG:6677
landuse = landuse.to_crs(epsg=32654) # I do this because I need a CRS that keeps information about distance to compute the buffers
unwanted = unwanted.to_crs(epsg=32654)
print(f"geometry attribute name: {landuse.geometry.name}") # this gives the name of the attibute corresponding to the geometry column (a GeoSeries)


original CRS: EPSG:6677
geometry attribute name: geometry


In [5]:
# filter only the parks from the land use dataframe
print(f"Total polygons in the land usage dataset: {landuse.shape[0]}")
parks = landuse[landuse["LU_1"] == 300].copy() # 300 identifies parks
print(f"Number of parks' polygons: {parks.shape[0]}")

# create a new index and update the areas
parks['park_id'] = range(1,len(parks)+1)
parks.set_index(parks.park_id)
parks.loc[:,'AREA'] = parks.geometry.area
parks = parks.rename(columns={'AREA':'area'})
initial_parks = parks.shape[0]

Total polygons in the land usage dataset: 815736
Number of parks' polygons: 14030


##### Remove unwanted features

In [6]:
# remove features that are mostly unwanted
overlap = gpd.overlay(parks, unwanted, how='intersection')
overlap['overlap_area'] = overlap.geometry.area
tot_overlap = overlap.groupby('park_id')['overlap_area'].sum().reset_index()
parks = parks.merge(tot_overlap, on='park_id', how='left')
parks['overlap_area'] = parks['overlap_area'].fillna(0)
parks['ov_percentage'] = (parks['overlap_area']/parks['area'])* 100
filtered_parks = parks[parks['ov_percentage'] <= 50]
parks_after_removal = filtered_parks.shape[0]
n_parks_removed1 = initial_parks - parks_after_removal
print(f"{n_parks_removed1} parks removed")

1546 parks removed


The following process dissolves and explodes the UGS. Therefore by design the average area of the parks will increase after the process.
By moving this process _after_ removing unwanted polygons, a higher number of unwanted polygons will be removed.

In [7]:
# merge the adjcent parks into a single entity
filtered_parks = filtered_parks.dissolve()
filtered_parks = filtered_parks.explode()
print(f"Number of parks' after merging: {filtered_parks.shape[0]}")
print(f"Step 1 eliminated {n_parks_removed1}")
print(f"Step 2 eliminated {parks_after_removal - filtered_parks.shape[0]}")
filtered_parks.loc[:,'area'] = filtered_parks.geometry.area # the value changed after merging

Number of parks' after merging: 10517
Step 1 eliminated 1546
Step 2 eliminated 1967


##### Include bodies of water

Main idea: a body of water should be included only if most of its perimeter touches a park. <br>
This would allow for ponds within parks to be included into the park itself, however huge rivers would be omitted. <br>
Some complications:
- Parks can be divided into multiple polygons, so it is important to consider that the total perimiter touching all polygons that compose the park is what matters to evaluate a bluespace's inclusion in the dataset.
- My solution is dissolving all parks in one single polygon. However bluespace that touches multiple parks may wrongly be added to the dataset.

HOWEVER: many smaller bodies of water are already included into the parks' dataset.

In [10]:
# Dissolve the parks into a single geometry (unified park boundary)
unipark = filtered_parks.dissolve()

# Create a single boundary geometry for the parks
park_boundary = unipark.geometry.boundary.union_all()  # This creates a single LineString

# Apply buffering to the park boundary (adjust buffer size as needed, e.g., 10 meters)
buffered_park_boundary = park_boundary.buffer(10) # I checked both 2 and 10 and I find 10 meters more suitable.

# Filter the bodies of water (LU_1 = 700) and ensure it's a copy
bluespace = landuse[landuse['LU_1'] == 700].copy()

# Calculate the boundary of each body of water
bluespace.loc[:, 'boundary'] = bluespace.geometry.boundary

# Intersect the buffered park boundary with the water feature boundaries
bluespace.loc[:, 'shared_boundary'] = bluespace['boundary'].apply(lambda b: b.intersection(buffered_park_boundary))

# Calculate the length of the shared boundary for each water feature
bluespace.loc[:, 'shared_length'] = bluespace['shared_boundary'].apply(lambda b: b.length)

# Calculate the percentage of the water feature's boundary that is shared with the buffered park boundary
bluespace.loc[:, 'perimeter'] = bluespace['boundary'].apply(lambda b: b.length)
bluespace.loc[:, 'shared_percentage'] = (bluespace['shared_length'] / bluespace['perimeter']) * 100

# Filter water features where at least 80% of the boundary is adjacent to the park boundary or its buffer
adj_bluespace = bluespace[bluespace['shared_percentage'] >= 80]

# Optional: Add these selected water features back to the parks dataset
parks_with_water = gpd.GeoDataFrame(pd.concat([filtered_parks, adj_bluespace], ignore_index=True))

parks_with_water = parks_with_water.drop(columns=['boundary', 'shared_boundary'])

print(f"Number of parks after joining the bodies of water: {parks_with_water.shape[0]}")

Number of parks after joining the bodies of water: 11160


#### Add missing greenspaces

By visualizing the data in QGIS it can be noted that shrines and temples' greenery is not included into the dataset. This could be an acceptable limitation to the dataset. However, I believe some areas particularly relevant areas should be included in the UGS dataset. <br>
Some examples are:
- Imperial Palace East National Gardens
- Meji Jingu Gyoen 

##### Add new polygons

In [14]:
new_parks_data = [
    {
        "name": "Imperial palace east naional gardens",
        "geometry": Polygon([
            (387234.5917162087, 3950150.3966461713),
            (387281.6385052205, 3950168.129666645),
            (387281.6385052205, 3950175.367634185),
            (387299.7334240712, 3950182.06275416),
            (387296.6572878666, 3950192.91970547),
            (387359.6276054669, 3950224.314389676),
            (387368.4941157038, 3950208.3908610875), 
            (387420.78843118215, 3950227.7524242587),
            (387424.22646576376, 3950215.266930252),
            (387469.1018645135, 3950223.7715421114),
            (387476.70173043077, 3950191.743535746),
            (387572.785749528, 3950213.909811337),
            (387577.95620084833, 3950253.8716067197),
            (387573.0173003257, 3950255.1743741767),
            (387577.77350422693, 3950275.066383232),
            (387581.29547903966, 3950274.404584143),
            (387581.8662668961, 3950276.1576717636),
            (387631.67757471907, 3950268.3064957964),
            (387637.2036606363, 3950244.309275006),
            (387753.7349380347, 3950238.5189009737),
            (387716.0975068253, 3950056.845915713), 
            (387751.5635477726, 3950052.5031351885),
            (387735.64001918404, 3949914.2579551693),
            (387653.1271892249, 3949929.457687004),
            (387571.3381560198, 3949601.57775743),
            (387418.6170409201, 3949622.5678632963), 
            (387418.6170409201, 3949646.091257802),
            (387380.97960971063, 3949696.0332338302), 
            (387391.83656102105, 3949709.7853721566), 
            (387197.1352341878, 3950023.189366651), 
            (387239.11544592143, 3950051.779338435), 
            (387213.05876277643, 3950118.3686398054), 
            (387240.5630394295, 3950130.6731846235), 
            (387234.5917162087, 3950150.3966461713)]),

        "area": None,  # Area will be calculated later
        "NAME1" : "千代田区"
    },
    {
        "name": "Kosuge west park",
        "geometry": Polygon([
            (393136.6516049383, 3957214.7029571617), 
            (393285.64975817315, 3957210.4108292903), 
            (393282.8564686064, 3957110.3974369955), 
            (393215.1362288645, 3957110.12492094), 
            (393214.18242267094, 3957088.868668627), 
            (393170.8523698784, 3957090.640022986), 
            (393172.8962402931, 3957166.126970304), 
            (393135.42528268945, 3957167.2170345252), 
            (393136.6516049383, 3957214.7029571617)
            ]),
        "area": None,
        "NAME1": "葛飾区", # compute later         
    },
    {
        "name": "Kosuge east sports park",
        "geometry": Polygon([
            (393470.2250258569, 3957303.2624996495),
            (393600.7602163453, 3957321.248559299),
            (393624.74162921164, 3957120.267968516),   
            (393493.66140661266, 3957104.4620373086), 
            (393470.2250258569, 3957303.2624996495)
            ]),
        "area": None,
        "NAME1": "葛飾区", # compute later         
    },
]

In [15]:
new_parks = gpd.GeoDataFrame(new_parks_data)
new_parks = new_parks.set_crs(epsg=32654)
new_parks = new_parks.to_crs(parks_with_water.crs)
new_parks["area"] = new_parks.geometry.area

##### Extract 'missclassified' parks

In [16]:
landuse.info() # AREA and CODE5 are numeric
misclassified_data = [
    {"area": 701833.77077, "CODE5": 13113032000, "NAME1": "渋谷区", "NAME2": "代々木神園町"},
    {"area": 12442.61194, "CODE5": 13120026004, "NAME1": "練馬区", "NAME2": "土支田四丁目"},
    {"area": 1617.28064, "CODE5": 13120026004, "NAME1": "練馬区", "NAME2": "土支田四丁目"},
    {"area": 26105.87451, "CODE5": 13115021001, "NAME1": "杉並区", "NAME2": "善福寺一丁目"},
    {"area": 16520.83983, "CODE5": 13119046002, "NAME1": "板橋区", "NAME2": "舟渡二丁目"},
    {"area": 21991.04371, "CODE5": 13117007002, "NAME1": "北区", "NAME2": "浮間二丁目"},
    {"area": 19574.51866, "CODE5": 13112019004, "NAME1": "世田谷区", "NAME2": "喜多見四丁目"},
    {"area": 30062.09595, "CODE5": 13112007002, "NAME1": "世田谷区", "NAME2": "岡本二丁目"},
    {"area": 3847.82543, "CODE5": 13112007002, "NAME1": "世田谷区", "NAME2": "岡本二丁目"},
    {"area": 3133.72299, "CODE5": 13112007002, "NAME1": "世田谷区", "NAME2": "岡本二丁目"},
    {"area": 35459.23117, "CODE5": 13111055002, "NAME1": "大田区", "NAME2": "南千束二丁目"},
    {"area": 1191.98565, "CODE5": 13111055002, "NAME1": "大田区", "NAME2": "南千束二丁目"},
    {"area": 817.15830, "CODE5": 13111055002, "NAME1": "大田区", "NAME2": "南千束二丁目"},
    {"area": 3360.63220, "CODE5": 13111055002, "NAME1": "大田区", "NAME2": "南千束二丁目"},
    {"area": 5317.68497, "CODE5": 13111026005, "NAME1": "大田区", "NAME2": "中央五丁目"},
    {"area": 4569.40623, "CODE5": 13111026005, "NAME1": "大田区", "NAME2": "中央五丁目"},
    {"area": 1638.38160, "CODE5": 13111026005, "NAME1": "大田区", "NAME2": "中央五丁目"}
]

misclassified_df = pd.DataFrame(misclassified_data)

landuse = landuse.rename(columns={"AREA":"area"})

matching_polygons = landuse.merge(
    misclassified_df,
    on=["area", "CODE5", "NAME1", "NAME2"],  # Match these columns
    how="inner"
)
 
print(f"Number of matches: {matching_polygons.shape[0]} out of {len(misclassified_data)}") 

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 815736 entries, 0 to 815735
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   AREA      815736 non-null  float64 
 1   LU_1      815736 non-null  int32   
 2   LU_2      815736 non-null  int32   
 3   LU_3      815736 non-null  int32   
 4   LU_4      815736 non-null  int32   
 5   CODE2     815736 non-null  int32   
 6   CODE3     815736 non-null  int32   
 7   CODE4     815736 non-null  int32   
 8   CODE5     815736 non-null  int64   
 9   NAME1     815736 non-null  object  
 10  NAME2     815505 non-null  object  
 11  geometry  815736 non-null  geometry
dtypes: float64(1), geometry(1), int32(7), int64(1), object(2)
memory usage: 52.9+ MB
Number of matches: 17 out of 17


##### Add new data to the park layer

In [33]:
final_parks = gpd.GeoDataFrame(pd.concat([parks_with_water, matching_polygons], ignore_index=True))
final_parks = gpd.GeoDataFrame(pd.concat([final_parks, new_parks], ignore_index=True))

##### Finalize the park layer
- Dissolve & Explode
- Recompute area
- Drop usless columns

In [34]:
# drop useless columns
final_parks = final_parks.loc[:, ["area","CODE5","NAME1","NAME2","geometry"]]
n_parks_after_addition = final_parks.shape[0]

# reduce number of parks by merging adjacent polygons
final_parks = final_parks.dissolve() #I need to group by name, but if I join just 134 instead of 745
final_parks = final_parks.explode()

# recompute area after merging polygons
final_parks['area'] = final_parks.geometry.area
f_n_parks = final_parks.shape[0]
print(f"The final number of parks is: {f_n_parks}")
print(f"Polygons assimilated by joining: {n_parks_after_addition - f_n_parks}")

final_parks["park_id"] = range(1, len(final_parks)+1)

The final number of parks is: 10451
Polygons assimilated by joining: 729


#### Merge park collections

This section is written a few weeks after the previous part of the notebook. I got rid of the to_file function, so there is not output being exported. For efficiency reasons I use as UGS layer the one present in the geopackage Tokyo_ugs_accessibility (which is produced by the notebook above). <br>
I noticed that some big parks (e.g., Ueno) are composed by multiple smaller park polygons. While this does not directly affect the accessibility measure critically, it will have consequences when I will compute the accessibility for different park size categories. <br>
While I thought about different potential solution, the most efficient seems to be the following:
- create a 5m buffer around parks
- dissolve the buffered layer parks
- explode the parks (from single to multiple polygons)
- create a negative 5m buffer
- recompute park areas

In [36]:
ugs = final_parks.copy()
print(f"I have {ugs.shape[0]} green spaces")
buff_ugs =  ugs.copy()
buff_ugs['geometry'] = buff_ugs.buffer(5) # If I do .buffer I get a geoseries

I have 10451 green spaces


In [37]:
buff_ugs = buff_ugs.dissolve(by='NAME1') # without "NAME1" I lose the information
merged_ugs = buff_ugs.explode()
merged_ugs['geometry'] = merged_ugs['geometry'].buffer(-5)


print("I buffer, dissolve, explode and debuffer")
print(f"After the procedure I have {merged_ugs.shape[0]} parks")

merged_ugs["park_id"] = range(1, len(merged_ugs) + 1)
merged_ugs['area'] = merged_ugs.geometry.area

I buffer, dissolve, explode and debuffer
After the procedure I have 8430 parks


In [22]:
# export UGS layer
# merged_ugs.to_file("../data/final/tokyo_ugs_accessibility.gpkg", layer='ugs')